In [10]:
import pandas as pd
import numpy as np

In [11]:
df_filtrado = pd.read_csv('pesajes-30-07-2021-06-08-2021_anonimizado.csv')
df_filtrado['fecha'] = pd.to_datetime(df_filtrado['fecha'])

def determinar_tipo_cartonero(row):
    """
    Si eventualmente hubiera otro tipo de cartonere devolvería no especificado.
    """
    if row['legacyId'].startswith('LE'):
        val = 'LE'   
    elif row['legacyId'] == 'No especificado':
        val = 'No especificado'
    else:
        val = 'RA'
    return val

df_filtrado['tipoCartonero'] = df_filtrado.apply(determinar_tipo_cartonero, axis=1)

In [12]:
df_filtrado

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material,tipoCartonero
0,2021-07-30,R8,2.0,NaN,24,COMPA40,2606,BARRACAS,Mezcla B,RA
1,2021-07-30,R8,4.0,NaN,15,COMPA47,2608,BARRACAS,Mezcla B,RA
2,2021-07-30,R8,4.0,NaN,21,COMPA47,2608,BARRACAS,Mezcla B,RA
3,2021-07-30,R8,4.0,NaN,24,COMPA47,2608,BARRACAS,Mezcla B,RA
4,2021-07-30,R8,5.0,NaN,35,COMPA112,6387,BARRACAS,Mezcla B,RA
...,...,...,...,...,...,...,...,...,...,...
3392,2021-08-05,E13,103.0,NaN,57,COMPA92,5515,SAAVEDRA,Mezcla B,RA
3393,2021-08-05,E13,152.0,NaN,61,COMPA1240,4972,SAAVEDRA,Mezcla B,RA
3394,2021-08-05,E13,161.0,NaN,43,COMPA1448,4973,SAAVEDRA,Mezcla B,RA
3395,2021-08-05,E13,203.0,NaN,15,COMPA2210,5546,SAAVEDRA,Mezcla B,RA


In [13]:
df_precios = pd.read_csv('precios_predio.csv') 

In [14]:
df_precios

,Sede,Material,Precio RA,Precio LE
0,AVELLANEDA,Mezcla B,25,15
1,AVELLANEDA,Papel Blanco B,20,20
2,AVELLANEDA,Vidrio B,30,40
3,AVELLANEDA,Chatarra,15,30
4,AVELLANEDA,TELA,35,25


### Funcion para calcular liquidacion total

In [15]:
def calcular_liquidacion_total(df_filtrado):
    df_filtrado_por_predios = df_filtrado.groupby(by=["predio","material","tipoCartonero"]).sum()["peso"].reset_index()
    
    suma_total = 0
    for index, row in df_filtrado_por_predios.iterrows():
        sede = row['predio']
        material = row['material']
        peso = row['peso']
        tipoCartonero = row['tipoCartonero']

        if tipoCartonero == 'LE':
            costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio LE'].values
        elif tipoCartonero == 'RA':
            costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio RA'].values

        if costo.size > 0:
            suma_total += peso*costo[0]
    
    return suma_total

--------------

In [16]:
df_filtrado_por_predios = df_filtrado.groupby(by=["predio","material","tipoCartonero"]).sum()["peso"].reset_index()

suma_total = 0
for index, row in df_filtrado_por_predios.iterrows():
    sede = row['predio']
    material = row['material']
    peso = row['peso']
    tipoCartonero = row['tipoCartonero']
    
    if tipoCartonero == 'LE':
        costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio LE'].values
    elif tipoCartonero == 'RA':
        costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio RA'].values

    if costo.size > 0:
        suma_total += peso*costo[0]

#df_filtrado_por_predios

In [17]:
df_filtrado_por_predios

,predio,material,tipoCartonero,peso
0,AVELLANEDA,TELA,LE,1831
1,AVELLANEDA,TELA,RA,2708
2,BARRACAS,Chatarra,LE,342
3,BARRACAS,Chatarra,RA,2199
4,BARRACAS,Mezcla B,LE,7493
5,BARRACAS,Mezcla B,RA,42456
6,BARRACAS,Papel Blanco B,LE,149
7,BARRACAS,Papel Blanco B,RA,1156
8,BARRACAS,Vidrio B,LE,511
9,BARRACAS,Vidrio B,RA,8112


In [32]:
df_filtrado_por_compa = df_filtrado.groupby(by=["predio","material","tipoCartonero","cartonero"]).sum()["peso"].reset_index()
#df_filtrado_por_compa

df_pago_total_por_compa = pd.DataFrame(df_filtrado_por_compa["cartonero"].unique(),columns=["compa"])
df_pago_total_por_compa["pago"] = [0 for k in range(0,len(df_pago_total_por_compa))]

for index, row in df_filtrado_por_compa.iterrows():
    sede = row['predio']
    material = row['material']
    peso = row['peso']
    tipoCartonero = row['tipoCartonero']
    carto = row["cartonero"]
    
    if tipoCartonero == 'LE':
        costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio LE'].values
    elif tipoCartonero == 'RA':
        costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio RA'].values

    if costo.size > 0:
        df_pago_total_por_compa.loc[df_pago_total_por_compa[df_pago_total_por_compa["compa"] == carto].index,"pago"] += peso*costo[0]
        #suma_total += peso*costo[0]
 

#df_filtrado_por_predios

#df_suma_total_por_compa
#df_filtrado_por_compa

In [33]:
df_pago_total_por_compa

,compa,pago
0,COMPA1116,1750
1,COMPA1718,15000
2,COMPA2286,3500
3,COMPA2330,10850
4,COMPA2371,1775
...,...,...
1067,COMPA931,0
1068,COMPA943,0
1069,COMPA981,0
1070,COMPA982,0


In [ ]:
def calcular_liquidacion_por_compa(df_filtrado,df_precios):
    df_filtrado_por_compa = df_filtrado.groupby(by=["predio","material","tipoCartonero","cartonero"]).sum()["peso"].reset_index()
    df_pago_total_por_compa = pd.DataFrame(df_filtrado_por_compa["cartonero"].unique(),columns=["compa"])
    df_pago_total_por_compa["pago"] = [0 for k in range(0,len(df_pago_total_por_compa))]

    for index, row in df_filtrado_por_compa.iterrows():
        sede = row['predio']
        material = row['material']
        peso = row['peso']
        tipoCartonero = row['tipoCartonero']
        carto = row["cartonero"]
    
        if tipoCartonero == 'LE':
            costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio LE'].values
        elif tipoCartonero == 'RA':
            costo = df_precios[(df_precios['Material'] == material) & (df_precios['Sede'] == sede)]['Precio RA'].values

        if costo.size > 0:
            df_pago_total_por_compa.loc[df_pago_total_por_compa[df_pago_total_por_compa["compa"] == carto].index,"pago"] += peso*costo[0]
    return df_pago_total_por_compa